In [1]:
#Importar librerias
from sklearn.cross_validation import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import column_or_1d

#Extraccion de la lista de ordenes de cabeceras seleccionadas
xlss = pd.read_excel('Process_considerations.xlsx', sheetname='FULL_DATASET_NC', 
                     header=None, index=False)
list_select = xlss.iloc[7,7].replace("c(","").replace(")","").split(",")
print(list_select)

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


['4:5', '7:8', '10', '12', '14', '16', '18', '20', '22', '24', '26', '28:29', '44:62', '77', '79', '81', '83', '85:98', '102:105', '107:112', '114:115', '117:119', '126:172', '183:185']


In [2]:
#En los casos de ordenes separados por ":" se toma los primeros
list_select = [_.split(":")[0] for _ in list_select]
print(list_select)

['4', '7', '10', '12', '14', '16', '18', '20', '22', '24', '26', '28', '44', '77', '79', '81', '83', '85', '102', '107', '114', '117', '126', '183']


In [3]:
#Extraccion de las cabeceras seleccionadas segun su numero indicado
matriz_head = xlss.iloc[:, range(2,6)].values
print(matriz_head[:10])

[[1 2 3 4]
 ['"ï..MES_CORTE"' '"CODDOC"                 "' 'CODUNICOCLI"            "'
  'LINEA_TC"']
 [5 6 7 8]
 ['"TIPO_FLUJO_TC"' '"CODSBS"' '"INGRESO_BRUTO"' '"EMP_REP_TC_SF"']
 [9 10 11 12]
 ['"EMP_REP_TC_IBK"' '"EMP_REP_PP_SF"' '"EMP_REP_PP_IBK"'
  '"EMP_REP_CONV_SF"']
 [13 14 15 16]
 ['"EMP_REP_CONV_IBK"' '"EMP_REP_VEH_SF"' '"EMP_REP_VEH_IBK"'
  '"EMP_REP_HIP_SF"']
 [17 18 19 20]
 ['"EMP_REP_HIP_IBK"' '"FLG_TC_SF"' '"FLG_TC_IBK"' '"FLG_PP_SF"']]


In [4]:
list_orden, list_head = [], []
for index in range(matriz_head.__len__()):
    if index % 2 == 0:
        list_orden.extend(matriz_head[index])
    else:
        list_head.extend(matriz_head[index])
del xlss
del matriz_head
print("list_orden: ", list_orden[:20])
print("list_head: ", list_head[:20])

list_orden:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
list_head:  ['"ï..MES_CORTE"', '"CODDOC"                 "', 'CODUNICOCLI"            "', 'LINEA_TC"', '"TIPO_FLUJO_TC"', '"CODSBS"', '"INGRESO_BRUTO"', '"EMP_REP_TC_SF"', '"EMP_REP_TC_IBK"', '"EMP_REP_PP_SF"', '"EMP_REP_PP_IBK"', '"EMP_REP_CONV_SF"', '"EMP_REP_CONV_IBK"', '"EMP_REP_VEH_SF"', '"EMP_REP_VEH_IBK"', '"EMP_REP_HIP_SF"', '"EMP_REP_HIP_IBK"', '"FLG_TC_SF"', '"FLG_TC_IBK"', '"FLG_PP_SF"']


In [5]:
#Limpio data vacia en la lista de cabeceras
list_head = [_.replace('"','') for _ in list_head if isinstance(_, str)]
list_head_select = [list_head[list_orden.index(_)] for _ in list_orden if str(_) in list_select]

print("list_head: ", list_head[:20])
print("list_head_select: ", list_head_select[:20])

list_head:  ['ï..MES_CORTE', 'CODDOC                 ', 'CODUNICOCLI            ', 'LINEA_TC', 'TIPO_FLUJO_TC', 'CODSBS', 'INGRESO_BRUTO', 'EMP_REP_TC_SF', 'EMP_REP_TC_IBK', 'EMP_REP_PP_SF', 'EMP_REP_PP_IBK', 'EMP_REP_CONV_SF', 'EMP_REP_CONV_IBK', 'EMP_REP_VEH_SF', 'EMP_REP_VEH_IBK', 'EMP_REP_HIP_SF', 'EMP_REP_HIP_IBK', 'FLG_TC_SF', 'FLG_TC_IBK', 'FLG_PP_SF']
list_head_select:  ['LINEA_TC', 'INGRESO_BRUTO', 'EMP_REP_PP_SF', 'EMP_REP_CONV_SF', 'EMP_REP_VEH_SF', 'EMP_REP_HIP_SF', 'FLG_TC_SF', 'FLG_PP_SF', 'FLG_CONV_SF', 'FLG_VEH_SF', 'FLG_HIP_SF', 'FLG_BANCARIZADO', 'SALDO_TC_SF', 'IND_EMP_REP_PP_SF', 'IND_EMP_REP_CONV_SF', 'IND_EMP_REP_VEH_SF', 'IND_EMP_REP_HIP_SF', 'SALDO_TOT_ACT', 'IND_VAR_OFERTA_UM', 'EDAD']


In [6]:
#Importar Data Set 
dataset = pd.read_csv("datos_tc_uni.csv")
list_head.clear()
list_select.clear()
#Importar Data Set con las cabeceras dentro del csv
list_head_select = [_ for _ in list_head_select if _ in dataset.loc[0,:].to_dict().keys()]
print("Cabeceras seleccionadas dentro del CSV")
print(list_head_select)
print("//////////////////////////////////////////////////")

Cabeceras seleccionadas dentro del CSV
['LINEA_TC', 'INGRESO_BRUTO', 'EMP_REP_PP_SF', 'EMP_REP_CONV_SF', 'EMP_REP_VEH_SF', 'EMP_REP_HIP_SF', 'FLG_TC_SF', 'FLG_PP_SF', 'FLG_CONV_SF', 'FLG_VEH_SF', 'FLG_HIP_SF', 'FLG_BANCARIZADO', 'SALDO_TC_SF', 'IND_EMP_REP_PP_SF', 'IND_EMP_REP_CONV_SF', 'IND_EMP_REP_VEH_SF', 'IND_EMP_REP_HIP_SF', 'SALDO_TOT_ACT', 'EDAD', 'CAMP_TOT', 'RECENCIA_CAMP', 'CREC_ENT_REP_TC']
//////////////////////////////////////////////////


In [7]:
X = dataset.loc[:,list_head_select].values
list_head_select.clear()
y = dataset.loc[:,["TARGET_XF"]].values

print(type(X))

<class 'numpy.ndarray'>


In [8]:
#Limpiar Data de las columnas del csv
list_nan = ["", "NaN", "Nan", "None", "nan"]
def cleann(x):
    return x if str(x) not in list_nan else 0

cleann = np.vectorize(cleann)
X = cleann(X)

In [9]:
#Dividimos nuestra set de Training y Testing
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25, random_state=0)

#Transformar a la misma escala
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train = sc_X.fit_transform(X_train.astype(float))
X_test = sc_X.transform(X_test.astype(float))

#Ajustamos Regresion logistica para el Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train.ravel())

#Exactitud:
print("Porcentaje de exactitud: ", classifier.score(X_test, y_test))

Porcentaje de exactitud:  0.762265153024


In [10]:
#Prediction del Test set
y_pred = classifier.predict(X_test)
y_pred_proba = classifier.predict_proba(X_test)
#Construir la Matriz de Confusion
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusion: ")
print(cm)

Matriz de Confusion: 
[[10241   803]
 [ 2778  1241]]


In [11]:
y_pred_proba.shape

(15063, 2)